### Init

In [2]:
import numpy as np
from collections import defaultdict
import string

In [3]:
with open("../inputs/input12.txt") as file: raw_data = file.read()
data = np.asarray([list(d) for d in raw_data.split("\n")], dtype=str)
temp_grid = np.pad(data, ((1,1),(1,1)), 'constant', constant_values = ".")
grid = np.empty((np.shape(temp_grid)), dtype=object)

### Custom Function(s)

In [5]:
class Tile:
    def __init__(self, _x, _y, _val):
        self.x = _x
        self.y = _y
        self.val = _val
        self.neighbours = []
        self.isAvailable = True
        self.sides = 4

    def __repr__(self):
        return str(self.val)

    def FindValidNeighbours(self, grid):
        for i in range(-1, 2):
            for j in range(-1, 2):
                if abs(i+j) == 1:
                    neighbour = grid[self.x+i, self.y+j]
                    if neighbour.val == self.val:
                        self.neighbours.append(neighbour)
                        self.sides -= 1

class Farm:
    def __init__(self, _id):
        self.id = _id
        self.plot = []
        self.area = 0
        self.perimeter = 0
        self.price = 0

    def __repr__(self):
        return str(self.id)

    def Calc(self):
        if not self.price:
            self.area = len(self.plot)
            cnt = 0
            for plot in self.plot:
                cnt += plot.sides
            self.perimeter = cnt
            self.price = self.area * self.perimeter
        
def DFS_FindRatings(graph, node):
    cnt = 0
    S, discovered = [], []
    S.append(node)
    while S:
        v = S.pop(-1)
        if v not in discovered:
            discovered.append(v)
            
            if v.isAvailable:
                for neighbour in v.neighbours:
                    S.append(neighbour)
                v.isAvailable = False
                
    return discovered

### Part 1

In [7]:
# Create tiles and populate grid
for i,row in enumerate(grid):
    for j,col in enumerate(row):
        grid[i,j] = Tile(i, j, temp_grid[i,j])

In [8]:
# Find neighbouring nodes for each plot (gridpoint)
for i,row in enumerate(grid):
    for j,col in enumerate(row):
        if grid[i,j].val in string.ascii_uppercase:
            grid[i,j].FindValidNeighbours(grid)

In [9]:
# A farm is a collection of plots (farm = polygon, plot = vertex)
farms = []
for i,row in enumerate(grid):
    for j,col in enumerate(row):
        if grid[i,j].val in string.ascii_uppercase and grid[i,j].isAvailable:
            plot = DFS_FindRatings(grid, grid[i,j])
            farm = Farm(grid[i,j].val)
            farm.plot = plot
            farms.append(farm)

cnt = 0
for farm in farms:
    farm.Calc()
    cnt += farm.price
cnt

1375574

### Part 2

In [11]:
cnt = []
for f in farms:
    farm = f.plot
    corner = 0
    for plot in farm:
        N = len(plot.neighbours)
        
        if N == 0:
            corner += 4
            
        elif N == 1:
            corner += 2
                    
        elif N >= 2:
            for c1i in range(N-1):
                for c2i in range(c1i+1, N):
                    c1 = np.array([plot.neighbours[c1i].x, plot.neighbours[c1i].y])
                    c2 = np.array([plot.neighbours[c2i].x, plot.neighbours[c2i].y])
                    if (c1 - c2)[0] != 0 and (c1 - c2)[1] != 0:
                        dx,dy = c1 + c2 - 2*np.array([plot.x,plot.y]) 
                        if N == 2:
                            corner += 1
                        if grid[plot.x+dx, plot.y+dy] not in farm:
                            corner += 1
    cnt.append(corner*len(farm))
sum(cnt)

830566